In [1]:
#Importing libraries 
import pandas as pd
import numpy as np 
from sklearn.preprocessing import  StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf 
import pickle as pkl 

In [3]:
#Loading Datasets
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#Dropping the columns

data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [8]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
#Converting Categorical Feature  --- Gender
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])


In [24]:
#Converting Categorical Feature--- Geography

onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [26]:
##Concating the onehot encoded Feature 
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [28]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [38]:
##Independent and Dependent Feature 
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [40]:
##train_test_split Dependent and Independent Feature
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [50]:
##Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [51]:
##Save the encoders and scaler

with open('onehot_encoder_geo.pkl','wb') as file :
    pkl.dump(onehot_encoder_geo,file)

with open('label_encoder_gender.pkl','wb') as file :
    pkl.dump(label_encoder_gender,file)

with open('scaler.pkl','wb') as file:
    pkl.dump(scaler,file)

## Training the problem statement ----ANN


In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [58]:
#Build Model 

models = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ##HiddenLayer_1
    Dense(32,activation='relu'),## HiddenLayer_2
    Dense(1) #OutputLayer
])

In [59]:
##Compile the models 
models.compile(optimizer='Adam',loss='mean_absolute_error',metrics=['mae'])

In [63]:
models.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
from tensorflow.keras.callbacks import  EarlyStopping,TensorBoard
import datetime 

In [66]:
##Setup tensorboard
log_dir = "regression_logs/fit/" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [67]:
##Setup Early Stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [68]:
##Training the model

history = models.fit(
    X_train,y_train,
    validation_data = (X_test,y_test),
    epochs = 100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 101159.4219 - mae: 101159.4219 - val_loss: 98172.5781 - val_mae: 98172.5781
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 100784.0078 - mae: 100784.0078 - val_loss: 96852.4375 - val_mae: 96852.4375
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 97774.1484 - mae: 97774.1484 - val_loss: 93473.1016 - val_mae: 93473.1016
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 94511.1172 - mae: 94511.1172 - val_loss: 87725.8047 - val_mae: 87725.8047
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 87029.0547 - mae: 87029.0547 - val_loss: 80006.1562 - val_mae: 80006.1562
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 79955.2891 - mae: 79955.2891 - val_loss: 71380.1562 - val_mae: 71380.1562
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 71543.7656 - mae: 71543.7656 - val_loss: 63370.3945 - val_mae: 63370.3945
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/ste

In [69]:
##TensorBoard 

%load_ext tensorboard

In [75]:
%tensorboard --logdir regression_logs


In [77]:
##Evaluate the model 
test_loss , test_mae = models.evaluate(X_test,y_test )
print(f"Test Loss : {test_loss} \nTest_mae  :{test_mae}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50768.7891 - mae: 50768.7891
Test Loss : 49891.79296875 
Test_mae  :49891.79296875


In [78]:
#Saving the models 
models.save('regression.keras')